In [1]:
import pandas as pd

input_ = pd.read_excel('./5.vehicle_1.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)

input_

,No,Part Number,Make,Model,Region,Submodel,Year,Aspiration,Cylinder,Displacement,Fuel,Liter,Drive Type,Position,Row
0,1,40-1000,Chevrolet,Camaro,MEX,Lujo,1998,Naturally Aspirated,V6,231cid,GAS,3.8L,RWD,Front,1
1,1,40-1000,Chevrolet,Camaro,MEX,Base,1998,Naturally Aspirated,V6,231cid,GAS,3.8L,RWD,Front,2
2,1,40-1000,Chevrolet,Camaro,USA,Z28 SS,1998,Naturally Aspirated,V8,350cid,GAS,5.7L,RWD,Front,3
3,1,40-1000,Chevrolet,Camaro,USA,Z28,1998,Naturally Aspirated,V8,350cid,GAS,5.7L,RWD,Front,4
4,1,40-1000,Chevrolet,Camaro,USA,Base,1998,Naturally Aspirated,V6,231cid,GAS,3.8L,RWD,Front,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330423,1401,85-4822,Volvo,XC60,MEX,T6 R-Design,2011,Turbocharged,L6,2953cc,GAS,3L,AWD,Rear,190
330424,1401,85-4822,Volvo,XC60,USA,3.2 R-Design,2011,Naturally Aspirated,L6,3192cc,GAS,3.2L,FWD,Rear,191
330425,1401,85-4822,Volvo,XC60,USA,3.2 R-Design,2011,Naturally Aspirated,L6,3192cc,GAS,3.2L,AWD,Rear,192
330426,1401,85-4822,Volvo,XC60,USA,T6 R-Design,2011,Turbocharged,L6,2953cc,GAS,3L,AWD,Rear,193


In [2]:
input_2 = input_[['Part Number']].drop_duplicates(keep='first', ignore_index=True)
input_2 = input_2.sort_values(by=['Part Number'],
                              ascending=[True],
                              ignore_index=True)

input_2

,Part Number
0,40-1000
1,40-10005
2,40-10006
3,40-10007
4,40-10009
...,...
1279,85-4603
1280,85-4809
1281,85-4810
1282,85-4817


In [3]:
from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Part Number'] == input_2.loc[i, 'Part Number']].copy().reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        make, model, engine, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], ' '.join([df_temp.loc[j, 'Liter'], df_temp.loc[j, 'Cylinder'], df_temp.loc[j, 'Displacement']]), df_temp.loc[j, 'Year']

        if make not in dict_vehicle.keys():
            dict_vehicle[make] = {model: {engine: [year]}}
        elif model not in dict_vehicle[make].keys():
            dict_vehicle[make][model] = {engine: [year]}
        elif engine not in dict_vehicle[make][model].keys():
            dict_vehicle[make][model][engine] = [year]
        else:
            if year not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            list_year = []
            for engine in dict_vehicle[make][model].keys():
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            for engine in dict_vehicle[make][model].keys():
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2.to_excel('./6.vehicle.xlsx', index=False)

input_2

Progress: 100%|██████████████████████████| 1284/1284 [00:26<00:00, 48.44it/s]


,Part Number,Vehicle W/o Engine,Vehicle W/ Engine
0,40-1000,Chevrolet Camaro 1993-1998\nPontiac Firebird 1...,Chevrolet Camaro 1993-1995 3.4L V6 207cid\nChe...
1,40-10005,Cadillac CTS 2008-2015\nCadillac Escalade 2015...,Cadillac CTS 2008-2014 3.6L V6 217cid\nCadilla...
2,40-10006,Buick Enclave 2008-2017,Buick Enclave 2008-2017 3.6L V6 217cid
3,40-10007,Chevrolet Malibu 2005-2007,Chevrolet Malibu 2005-2007 2.2L L4 134cid\nChe...
4,40-10009,Chevrolet Suburban 1500 2000\nChevrolet Suburb...,Chevrolet Suburban 1500 2000 5.3L V8 325cid\nC...
...,...,...,...
1279,85-4603,Honda Passport 1998-2002\nIsuzu Rodeo 1998-2004,Honda Passport 1998-2002 3.2L V6 3165cc\nIsuzu...
1280,85-4809,Volvo V70 2005-2007\nVolvo XC70 2005-2008,Volvo V70 2005-2007 2.4L L5 2435cc\nVolvo V70 ...
1281,85-4810,Volvo XC90 2003-2014,Volvo XC90 2003-2006 2.5L L5 2521cc\nVolvo XC9...
1282,85-4817,Volvo XC90 2004-2014,Volvo XC90 2004-2006 2.5L L5 2521cc\nVolvo XC9...
